In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [3]:
df= pd.read_csv('../input/heart-failure-prediction/heart.csv')
df.head(10)
df.tail(10)

FileNotFoundError: [Errno 2] No such file or directory: '../input/heart-failure-prediction/heart.csv'

## Checking Unique Features in Qualitative Data
Before we proceed to feature engineering, we need to see what kind of categorical data that we have. Thus, checking unique features is necessary

In [ ]:
for column_names in df.columns:
    if df[column_names].dtypes==object:
        print(column_names+' has ')
        print(df[column_names].unique())

## Checking Quantitative Data
It is the same as previous, we need to see the statistical data to determine the null values or outliers

In [ ]:
for column_names in df.columns:
    if df[column_names].dtypes!=object:
        print(column_names+' has ')
        print(df[column_names].describe())

In [ ]:
df.isnull().sum()

In [ ]:
for column_names in df.columns:
    if df[column_names].max()!=1 and (df[column_names].dtypes=='float64' or df[column_names].dtypes=='int64'):
        print(column_names)

## Data Visualization

In [ ]:
for column_names in df.columns:
    if df[column_names].max()!=1 and (df[column_names].dtypes=='float64' or df[column_names].dtypes=='int64'):
        sns.displot(x=df[column_names], kde=True)

In [ ]:
sns.boxplot(x=df['Age'])

In [ ]:
sns.boxplot(x=df['RestingBP'])

In [ ]:
sns.boxplot(x=df['Cholesterol'])

In [ ]:
sns.boxplot(x=df['MaxHR'])

In [ ]:
sns.boxplot(x=df['Oldpeak'])

## Feature Engineering
Since we need to consider the outliers would affect the model or not, we need to see how much the outliers are in certain categories. Then, we need to consider replacing it

In [ ]:
Q1=df.quantile(0.25)
Q3=df.quantile(0.75)
IQR=Q3-Q1
numerical_value=['Age','RestingBP','Cholesterol','MaxHR','Oldpeak']
((df[numerical_value] < (Q1 - 1.5 * IQR)) | (df[numerical_value] > (Q3 + 1.5 * IQR))).sum()

In [ ]:
print(df[numerical_value].skew())

In [ ]:
print(df['Oldpeak'].quantile(0.1))
print(df['Oldpeak'].quantile(0.9))

In [ ]:
Q1= df.quantile(0.1)
Q3=df.quantile(0.9)
IQR=Q3-Q1
df["Oldpeak"] = np.where(df["Oldpeak"] <0.0, 0.0,df['Oldpeak'])
df["Oldpeak"] = np.where(df["Oldpeak"] >2.3, 2.3,df['Oldpeak'])
print(df['Oldpeak'].skew())

In [ ]:
label=LabelEncoder()
category_data=['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']
for word in category_data:
    df[word]=label.fit_transform(df[word])

In [ ]:
df.head()

In [ ]:
df.describe()

In [ ]:
scaled=MinMaxScaler()
need_to_scaled=['Age','ChestPainType','RestingBP','Cholesterol','RestingECG','MaxHR','Oldpeak','ST_Slope']

df[need_to_scaled]=scaled.fit_transform(df[need_to_scaled])

## Splitting Data Test and Train

In [ ]:
x=df.drop(columns='HeartDisease')
y=df['HeartDisease']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y, random_state=123,test_size=0.3)

In [ ]:
y_train

## Machine Learning

In [ ]:
lsvc= LinearSVC()
lsvc.fit(x_train,y_train)
score = lsvc.score(x_train, y_train)
print("Score: ", score)

In [ ]:
y_pred = lsvc.predict(x_test)
cr = classification_report(y_test, y_pred)
print(cr)